# 📊 Análise de Resultados — PySpark + Delta Lake

Este notebook tem como objetivo realizar análises exploratórias e agregações a partir dos dados estruturados na camada **Trusted** do Data Warehouse, utilizando **PySpark** em conjunto com o **Delta Lake**. Serão exploradas informações provenientes de tabelas fato e dimensões, permitindo insights sobre o perfil das empresas, distribuição geográfica, porte, natureza jurídica, entre outros aspectos relevantes.

## ⚙️ 1. Configuração Spark & Importes

In [36]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [37]:
builder = SparkSession.builder \
    .appName("App analise de dados") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

## 📥 2. Leitura da tabela Delta com explicação e contagem

In [ ]:
try:    
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../TRS/fato_cnpj")
    df: DataFrame = deltaTable.toDF()
    print(f"Total registros lidos: {df.count()}")
    df.show(5, truncate=False)
    df.printSchema()
except Exception as e:
    print(f"Erro na leitura da tabela Delta: {e}")

Total registros lidos: 4753435
+---------+----------+-------+-------------+-------------------------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+---------------------+------+-----------+------+--------+---+----------------+----+---------+----+---------+-------+--------+-------------------------+-----------------+----------------------+----------------------------------------+-----------------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+---------------------------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                  |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_f

## 🧼 3. Formatação das colunas para exibição

In [ ]:
def formatar_colunas(df: DataFrame) -> DataFrame:
    """
    Formata e renomeia colunas de um DataFrame com dados de empresas, tornando-o mais legível
    e pronto para visualizações ou análises exploratórias.

    Operações realizadas:
    - Concatenação dos campos do CNPJ (base, ordem e dígito) em uma única coluna.
    - Formatação de datas no padrão "dd/MM/yyyy".
    - Renomeação de colunas para nomes mais amigáveis (sem espaços, com letras maiúsculas).
    - Criação de colunas compostas como endereço completo.
    
    Args:
        df (DataFrame): DataFrame original contendo os dados da camada Trusted do CNPJ.

    Returns:
        DataFrame: Novo DataFrame com colunas formatadas e renomeadas para fins de exibição.
    """
    return df.select(
        f.concat_ws("", df.cnpj_base, df.cnpj_ordem, df.cnpj_dv).alias("CNPJ"),
        df.nome_fantasia.alias("NOME_FANTASIA"),
        df.situacao_cadastral.alias("SITUACAO_CADASTRAL"),
        f.date_format(df.data_situacao_cadastral, "dd/MM/yyyy").alias("DATA_SITUACAO_CADASTRAL"),
        df.motivo_situacao_cadastral.alias("SITUACAO_CADASTRAL_MOTIVO"),
        df.nome_cidade_exterior.alias("CIDADE_EXTERIOR"),
        df.codigo_pais.alias("CODIGO_PAIS"),
        f.date_format(df.data_inicio_atividade, "dd/MM/yyyy").alias("DATA_INICIO_ATIVIDADE"),
        df.cnae_fiscal.alias("CNAE_PRINCIPAL"),
        df.cnae_fiscal_secundaria.alias("CNAES_SECUNDARIOS"),
        f.concat_ws(" ", df.tipo_logradouro, df.logradouro).alias("LOGRADOURO"),
        df.numero.alias("NUMERO"),
        df.complemento.alias("COMPLEMENTO"),
        df.bairro.alias("BAIRRO"),
        df.cep.alias("CEP"),
        df.municipio.alias("MUNICIPIO"),
        df.uf.alias("UF"),
        df.telefone1.alias("TELEFONE"),
        df.email.alias("EMAIL"),
        df.descricao_cnae.alias("DESCRICAO_CNAE"),
        df.capital_social.alias("CAPITAL_SOCIAL"),
        df.razao_social.alias("RAZAO_SOCIAL"),
        df.codigo_natureza_juridica.alias("CODIGO_NATUREZA_JURIDICA"),
        df.descricao_natureza_juridica.alias("DESCRICAO_NATUREZA_JURIDICA"),
        df.codigo_porte_empresa.alias("CODIGO_PORTE"),
        df.opcao_simples.alias("OPCAO_PELO_SIMPLES"),
        f.date_format(df.data_opcao_simples, "dd/MM/yyyy").alias("DATA_OPCAO_SIMPLES"),
        f.date_format(df.data_exclusao_simples, "dd/MM/yyyy").alias("DATA_EXCLUSAO_SIMPLES"),
        df.opcao_mei.alias("OPCAO_MEI"),
        f.date_format(df.data_opcao_mei, "dd/MM/yyyy").alias("DATA_OPCAO_MEI"),
        f.date_format(df.data_exclusao_mei, "dd/MM/yyyy").alias("DATA_EXCLUSAO_MEI")
    )

df_formatado: DataFrame = formatar_colunas(df)
df_formatado.show(2, truncate=False)

+--------------+-------------------------------+------------------+-----------------------+-------------------------+---------------+-----------+---------------------+--------------+-----------------+-------------------------+------+-----------+------+--------+---------------+---+--------+-----------------+----------------------------------------+--------------+------------+------------------------+---------------------------+------------+------------------+------------------+---------------------+---------+--------------+-----------------+
|CNPJ          |NOME_FANTASIA                  |SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|SITUACAO_CADASTRAL_MOTIVO|CIDADE_EXTERIOR|CODIGO_PAIS|DATA_INICIO_ATIVIDADE|CNAE_PRINCIPAL|CNAES_SECUNDARIOS|LOGRADOURO               |NUMERO|COMPLEMENTO|BAIRRO|CEP     |MUNICIPIO      |UF |TELEFONE|EMAIL            |DESCRICAO_CNAE                          |CAPITAL_SOCIAL|RAZAO_SOCIAL|CODIGO_NATUREZA_JURIDICA|DESCRICAO_NATUREZA_JURIDICA|CODIGO_PORTE|OPCAO_PELO

## 📈 4. Análise de Abertura de Empresas

- Quantidade de empresas abertas por ANO e MÊS, ordenado de forma decrescente:

In [ ]:
df_empr_inicio_ano_mes: DataFrame = df.groupBy(
    f.year("data_inicio_atividade").alias("ANO"),
    f.month("data_inicio_atividade").alias("MES")
    ).agg(
        f.count("*").alias("EMPRESAS_ABERTAS")
    ).orderBy("ANO", "MES", ascending=False)

df_empr_inicio_ano_mes.select("EMPRESAS_ABERTAS", "ANO", "MES").show(10,truncate=False)

+----------------+----+---+
|EMPRESAS_ABERTAS|ANO |MES|
+----------------+----+---+
|19784           |2025|6  |
|42544           |2025|5  |
|41477           |2025|4  |
|42035           |2025|3  |
|45271           |2025|2  |
|58368           |2025|1  |
|26879           |2024|12 |
|34031           |2024|11 |
|39439           |2024|10 |
|37321           |2024|9  |
+----------------+----+---+
only showing top 10 rows


## 🏷️ 5. Status Cadastral das Empresas

- Criação da coluna `DESCRICAO_STATUS_EMPRESA`, com rótulos legíveis:

In [ ]:
df_status_empresa: DataFrame = df.withColumn(
    "DESCRICAO_STATUS_EMPRESA",
    f.when(
        df.situacao_cadastral == f.lit('01'),
        "ATIVA"
    ).when(
        df.situacao_cadastral.isin('02', '03', '04'),
        "INATIVA"
    ).otherwise("PENDENCIAS")
)

- Agrupamento por status:

In [47]:

# Recuperar a quantidade da coluna "DESCRICAO_STATUS_EMPRESA"
df_qtd_status: DataFrame = df_status_empresa.groupBy("DESCRICAO_STATUS_EMPRESA").agg(
    f.count("*").alias("QTD_EMPRESAS")
)

# Exibir resultados
df_qtd_status.show()
df_qtd_status.describe().show()

+------------------------+------------+
|DESCRICAO_STATUS_EMPRESA|QTD_EMPRESAS|
+------------------------+------------+
|              PENDENCIAS|     2260668|
|                 INATIVA|     2484499|
|                   ATIVA|        8268|
+------------------------+------------+



+-------+------------------------+------------------+
|summary|DESCRICAO_STATUS_EMPRESA|      QTD_EMPRESAS|
+-------+------------------------+------------------+
|  count|                       3|                 3|
|   mean|                    NULL|1584478.3333333333|
| stddev|                    NULL|1369618.3192117186|
|    min|                   ATIVA|              8268|
|    max|              PENDENCIAS|           2484499|
+-------+------------------------+------------------+



## 🧩 6. Distribuição por CNAE (Principal e Secundário)

In [48]:
df_cnae_distribuicao: DataFrame = df.select(
    df.cnae_fiscal.alias("CNAE_PRINCIPAL"), 
    df.descricao_cnae.alias("DESCRICAO_CNAE_PRINCIPAL"), 
    df.cnae_fiscal_secundaria.alias("CNAE_SECUNDARIO")
    ).filter(f.col("CNAE_SECUNDARIO").isNotNull())

df_cnae_distribuicao.show(truncate=False)

+--------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|CNAE_PRINCIPAL|DESCRICAO_CNAE_PRINCIPAL                                                                                           |CNAE_SECUNDARIO                                                                                                                                        |
+--------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|2229399       |Fabricação de artefatos de material plástico para outros usos não especificados anteriormente                      |3292202,32990

## 📊 7. Estatísticas Gerais

- Listar UFs distintas

In [49]:
df.select("uf").distinct().show()

+---+
| uf|
+---+
| SP|
| MG|
| RJ|
| PR|
| RS|
| BA|
| SC|
| GO|
| PE|
| CE|
| MT|
| ES|
| PA|
| DF|
| MA|
| PB|
| MS|
| RN|
| AM|
| PI|
+---+
only showing top 20 rows


- Contagem de empresas por UF

In [50]:
df_contagem_uf_empresas = df.groupBy("uf").agg(
    f.count("*").alias("QTD_EMPRESAS")
).orderBy("QTD_EMPRESAS", ascending=False)

df_contagem_uf_empresas.show()

+---+------------+
| uf|QTD_EMPRESAS|
+---+------------+
| SP|     1379898|
| MG|      513667|
| RJ|      382888|
| RS|      344456|
| PR|      326165|
| BA|      237630|
| SC|      231308|
| GO|      169298|
| PE|      141706|
| CE|      134480|
| ES|       96212|
| PA|       93181|
| MT|       91885|
| DF|       79580|
| MA|       72871|
| MS|       63846|
| PB|       59851|
| RN|       53391|
| AM|       51203|
| AL|       45296|
+---+------------+
only showing top 20 rows


- Estatísticas descritivas (Ex: capital social)

In [51]:
df.describe("capital_social").show()

+-------+------------------+
|summary|    capital_social|
+-------+------------------+
|  count|            486655|
|   mean|   11290841.310325|
| stddev|7.89383137213955E8|
|    min|              0.00|
|    max|   500000352182.73|
+-------+------------------+



## 🔍 8. Consultas Relevantes para Insights

- Top 10 Municípios com Mais Empresas

In [ ]:
df_municipio_por_uf: DataFrame = df.groupBy(df.municipio, df.uf).agg(
    f.count("*").alias("Qtd Empresas"),
).orderBy("Qtd Empresas", ascending=False)

df_municipio_por_uf.show(10)

+--------------+---+------------+
|     municipio| uf|Qtd Empresas|
+--------------+---+------------+
|     SAO PAULO| SP|      430363|
|RIO DE JANEIRO| RJ|      168441|
|BELO HORIZONTE| MG|       91174|
|      BRASILIA| DF|       79580|
|      CURITIBA| PR|       75285|
|      SALVADOR| BA|       66713|
|  PORTO ALEGRE| RS|       62352|
|     FORTALEZA| CE|       62262|
|       GOIANIA| GO|       51450|
|        RECIFE| PE|       42108|
+--------------+---+------------+
only showing top 10 rows


- Distribuição por Natureza Jurídica

In [58]:
df_empresas_por_nj = df.groupBy(
    df.descricao_natureza_juridica,
    df.codigo_natureza_juridica
).agg(
    f.count("*").alias("QTD_EMPRESAS")
).filter(
    df["codigo_natureza_juridica"].isNotNull()
).orderBy("QTD_EMPRESAS", ascending=False)

df_empresas_por_nj.select(
    df.descricao_natureza_juridica.alias("DESCRICAO_NATUREZA_JURIDICA"),
    df.codigo_natureza_juridica.alias("COD_NATUREZA_JURIDICA"),
    "QTD_EMPRESAS"
).show(truncate=False)

+------------------------------------------------------------------------+---------------------+------------+
|DESCRICAO_NATUREZA_JURIDICA                                             |COD_NATUREZA_JURIDICA|QTD_EMPRESAS|
+------------------------------------------------------------------------+---------------------+------------+
|Sociedade Empresária Limitada                                           |2062                 |182094      |
|Empresário (Individual)                                                 |2135                 |156222      |
|Candidato a Cargo Político Eletivo                                      |4090                 |44499       |
|Produtor Rural (Pessoa Física)                                          |4120                 |33767       |
|Associação Privada                                                      |3999                 |29262       |
|Sociedade Simples Limitada                                              |2240                 |13566       |
|Empresa D

## Delta Table - Controle de versões

In [ ]:
# Tabelas Trusted
deltaTable_estabelecimentos: DeltaTable = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
deltaTable_empresas: DeltaTable = DeltaTable.forPath(spark, "../TRS/empresas")
deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
deltaTable_natureza_juridica: DeltaTable = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
deltaTable_simples_nacional: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")

# Tabela Fato
deltaTable.history().show(vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------
 version             | 0                                                                                                                     
 timestamp           | 2025-07-20 15:54:41.123                                                                                               
 userId              | NULL                                                                                                                  
 userName            | NULL                                                                                                                  
 operation           | WRITE                                                                                                                 
 operationParameters | {mode -> Overwrite, partitionBy -> ["uf"]}                                                                            
 job  

In [51]:
def titulo(texto):
    print(f"{"=-"*20} >> {texto} << {"=-"*20}")

titulo("HISTÓRICO ESTABELECIMENTOS")
deltaTable_estabelecimentos.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO EMPRESAS")
deltaTable_empresas.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO MUNICÍPIOS RECEITA FEDERAL")
deltaTable_municipios_rf.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO CNAE")
deltaTable_cnae.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO NATUREZA JURÍDICA")
deltaTable_natureza_juridica.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO SIMPLES NACIONAL")
deltaTable_simples_nacional.history().show(vertical=True, truncate=False)

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- >> HISTÓRICO ESTABELECIMENTOS << =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 17                                                                                                                                                                                                                                                                                                                                                        
 timestamp           | 2025-07-20 14:31:32.288                                                                                                   

## Finalizar spark

In [112]:
spark.stop()